<a href="https://colab.research.google.com/github/wsGit7/Invest/blob/main/pdfHighlight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 40.6 MB/s eta 0:00:00


In [20]:
# super
import os
import re
import fitz  # PyMuPDF

def normalize_word(w):
    """
    Lowercase the word and remove punctuation from the beginning and end.
    """
    return re.sub(r'^\W+|\W+$', '', w.lower())

# --- Read the target sequences (each line is a full phrase) ---
with open("unikalneWyr.txt", "r", encoding="utf-8") as f:
    # Each non-empty line is a target phrase (sequence of words)
    target_lines = [line.strip() for line in f if line.strip()]

# Convert each target phrase into a list of normalized words.
target_sequences = []
for line in target_lines:
    words = line.split()  # split on whitespace
    normalized = [normalize_word(word) for word in words]
    target_sequences.append(normalized)

# --- Folder containing the PDF files ---
folder = "jurek"

# Process each PDF file in the folder.
for filename in os.listdir(folder):
    if not filename.lower().endswith(".pdf"):
        continue

    filepath = os.path.join(folder, filename)
    print(f"Processing {filepath}...")
    doc = fitz.open(filepath)

    for page in doc:
        # Get words from the page.
        # Each element is a tuple:
        # (x0, y0, x1, y1, word, block_no, line_no, word_no)
        page_words = page.get_text("words")
        # Create a list of (normalized_word, bounding_box) for each word.
        normalized_words = []
        for winfo in page_words:
            word_text = winfo[4]
            norm = normalize_word(word_text)
            bbox = fitz.Rect(winfo[0], winfo[1], winfo[2], winfo[3])
            normalized_words.append((norm, bbox))

        # For each target sequence, try to find a contiguous match in the page's words.
        for target_seq in target_sequences:
            t_len = len(target_seq)
            if t_len == 0:
                continue

            # Loop over positions in the normalized_words list.
            i = 0
            while i <= len(normalized_words) - t_len:
                match_found = True
                # Check sequentially if the target matches
                for j in range(t_len):
                    if normalized_words[i+j][0] != target_seq[j]:
                        match_found = False
                        break
                if match_found:
                    # Combine bounding boxes for the sequence.
                    rects = [normalized_words[i+j][1] for j in range(t_len)]
                    combined_rect = rects[0]
                    for r in rects[1:]:
                        combined_rect |= r  # union of rectangles
                    # Highlight the found sequence (yellow highlight).
                    annot = page.add_highlight_annot(combined_rect)
                    annot.set_colors(stroke=(1, 1, 0))  # Yellow: RGB (1,1,0)
                    annot.update()
                    # Advance the index to skip over this found sequence to avoid overlapping matches.
                    i += t_len
                else:
                    i += 1

    # Save the modified PDF with a new name.
    new_filepath = os.path.join(folder, "highlighted_" + filename)
    doc.save(new_filepath, garbage=4, deflate=True, clean=True)
    doc.close()
    print(f"Saved highlighted version as: {new_filepath}")


Processing jurek/ustawaSluzbieZagrani.pdf...
Saved highlighted version as: jurek/highlighted_ustawaSluzbieZagrani.pdf
